In [2]:
# --- IMPORTS ---
import numpy as np
import pandas as pd
import ast
import pickle
import bz2
import os

# NLP and Machine Learning libraries
import nltk
from nltk.stem.porter import PorterStemmer
# UPGRADE: Swapped CountVectorizer for TfidfVectorizer for much higher accuracy
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.metrics.pairwise import cosine_similarity

# --- LOAD & CLEAN DATA ---
print("Loading datasets from content-based-datasets folder...")
movies = pd.read_csv('datasets/content-based-datasets/tmdb_5000_movies.csv')
credits = pd.read_csv('datasets/content-based-datasets/tmdb_5000_credits.csv') 

# Merge on 'title'
movies = movies.merge(credits, on='title')
movies = movies[['movie_id', 'title', 'overview', 'genres', 'keywords', 'cast', 'crew']]

# Drop missing values early to prevent errors and save memory
movies.dropna(inplace=True)
movies.reset_index(drop=True, inplace=True)

# --- FEATURE EXTRACTION ---
def extract_features(text, feature_type='general', limit=None):
    """Parses stringified JSON, extracts names, and removes spaces."""
    extracted = []
    try:
        for i in ast.literal_eval(text):
            if feature_type == 'director':
                if i['job'] == 'Director':
                    extracted.append(i['name'].replace(" ", ""))
                    break 
            else:
                extracted.append(i['name'].replace(" ", ""))
            
            if limit and len(extracted) == limit:
                break
    except Exception:
        pass
    return extracted

print("Extracting features and applying weights...")
movies['genres'] = movies['genres'].apply(extract_features)
movies['keywords'] = movies['keywords'].apply(extract_features)
movies['cast'] = movies['cast'].apply(lambda x: extract_features(x, limit=3))
movies['crew'] = movies['crew'].apply(lambda x: extract_features(x, feature_type='director'))
movies['overview'] = movies['overview'].apply(lambda x: x.split())

# --- UPGRADE: FEATURE WEIGHTING ---
# WHY: A matching Director or Genre is a stronger signal than a matching plot word.
# HOW: By multiplying the lists, we duplicate the tags. TF-IDF will see "ChristopherNolan" 
# three times, mathematically forcing the algorithm to prioritize movies he directed.
movies['tags'] = (
    movies['overview'] + 
    (movies['genres'] * 2) +    # Double weight for genres
    movies['keywords'] + 
    (movies['cast'] * 2) +      # Double weight for top 3 actors
    (movies['crew'] * 3)        # Triple weight for the director
)

new_df = movies[['movie_id', 'title', 'tags']].copy()
new_df['tags'] = new_df['tags'].apply(lambda x: " ".join(x).lower())

# --- STEMMING ---
ps = PorterStemmer()
new_df['tags'] = new_df['tags'].apply(lambda text: " ".join([ps.stem(word) for word in text.split()]))

# --- UPGRADE: TF-IDF VECTORIZATION ---
print("Vectorizing text using TF-IDF and calculating similarity...")
# WHY: TF-IDF reduces the noise of common plot words and highlights highly specific tags.
tfidf = TfidfVectorizer(max_features=5000, stop_words='english')
vector = tfidf.fit_transform(new_df['tags']).toarray()

# Calculate Cosine Similarity
similarity = cosine_similarity(vector)

# --- RECOMMENDATION LOGIC & EXPORT ---
def recommend(movie):
    try:
        movie_index = new_df[new_df['title'] == movie].index[0]
        distances = sorted(list(enumerate(similarity[movie_index])), reverse=True, key=lambda x: x[1])[1:6]
        
        print(f"\nTest Recommendations for '{movie}':")
        for i in distances:
            print("-", new_df.iloc[i[0]].title)
    except IndexError:
        print("Movie not found in database.")

# Test the upgraded function
recommend('Batman Begins')

print("\nExporting models...")
os.makedirs('models/content-based-models', exist_ok=True)

pickle.dump(new_df, open('models/content-based-models/movie_list.pkl', 'wb'))

similarity_shrunk = similarity.astype(np.float16)

with bz2.BZ2File('models/content-based-models/similarity.pkl.bz2', 'wb') as f:
    pickle.dump(similarity_shrunk, f)

print("✅ Success! Upgraded Content-Based models saved to 'models/content-based-models/'.")

Loading datasets from content-based-datasets folder...
Extracting features and applying weights...
Vectorizing text using TF-IDF and calculating similarity...

Test Recommendations for 'Batman Begins':
- The Dark Knight Rises
- The Dark Knight
- The Prestige
- Insomnia
- Batman

Exporting models...
✅ Success! Upgraded Content-Based models saved to 'models/content-based-models/'.
